# Importing Packages

In [1]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import (
    StreamingStdOutCallbackHandler
)
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
from langchain.vectorstores import Chroma

# Loading Embeddings Engine

In [4]:
# embedding engine
hf_embedding = HuggingFaceInstructEmbeddings()

C:\Users\Administrator\anaconda3\envs\autogptq\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


# Initialize Knowledge base Resources

In [5]:
%%time
knowledge_base_loader = PyPDFDirectoryLoader("pdfs")
knowledge_base = knowledge_base_loader.load()

CPU times: total: 7min 4s
Wall time: 7min 9s


In [6]:
knowledge_base_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
knowledge_base_texts = knowledge_base_text_splitter.split_documents(knowledge_base)
len(knowledge_base_texts)

25364

# Saving Knowledge base

In [7]:
%%time
db = FAISS.from_documents(knowledge_base_texts, hf_embedding)

# save embeddings in local directory
db.save_local("faiss_knowledge_base")

CPU times: total: 21min 2s
Wall time: 20min 45s


# Loading Local Knowledge Base

In [49]:
# load from local
db = FAISS.load_local("faiss_patient_data_with_resources_1.2/", embeddings=hf_embedding)

# Loading Model

In [50]:
template = """Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [51]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [8]:
llm = LlamaCpp(
    # model_path="llama-2-7b-chat.ggmlv3.q2_K.bin",
    model_path="C:/Users/Alaa AI/Python Projects/Chatbots/GPT by me/llama-2-7b-chat.Q4_K_M.gguf",
    n_ctx=40000,
    n_gpu_layers=512,
    n_batch=30,
    callback_manager=callback_manager,
    temperature = 0.1,
    max_tokens = 30000,
    n_parts=1,
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [19]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Create a Query

In [42]:
query = "what is the date of birth of the examinee 'O'"
search = db.similarity_search(query, k=2)

In [55]:
query = " As an Experienced Doctor Give me an organized detailed report form about the examinee 'Ahmed Khaleel' : which contains Examinee Data in Details , Diagnostic Scale Results in Details and as an experienced doctor , Executive Function Scale Results in Details and as an experienced doctor in Details and as an experienced doctor, Learning Disability Assessment in Details and as an experienced doctor, Interview and case History Insights in details, perliminary Diagnosis in Details and as an experienced doctor, Proposed Intervention program in details, Medication Suggestions with doses and considerations for medication management, Professional Review Notes in details and as an experienced Doctor"
# query = "based on provided context Give me Data about the Examinee 'O' details Give me in organized form The Diagnostic Scale Results as number 1 , Executive Function Scale Results as number 2 , learning Disability assessment as number 3 , interview and case history insights as number 4 , preliminary diagnosis as number 5 , proposed intervention program as number 6 , medication suggestions as number 7 , and professional review notes as number 8"
# query  = "Give me a medication suggestions in an organized form with details for each medic which will help the examinee 'O' to be good and recover with doses and side effects as an experienced doctor in details"
search = db.similarity_search(query, k=2)

# Creating Prompt Template

In [56]:
# template = '''Context: {context}

# Based on Context provide me answer for following question
# Question: {question}

# Tell me the information about the fact. The answer should be from context only
# do not use general knowledge to answer the query'''
template = '''Context: {context}

Based on Context provide me answer for following question
Question: {question}

Tell me the information about the fact. The answer should be from context
use general knowledge to answer the query'''
prompt = PromptTemplate(input_variables=["context", "question"], template= template)
final_prompt = prompt.format(question=query, context=search)

In [57]:
llm_chain.run(final_prompt)

Llama.generate: prefix-match hit


 Based on the provided context, here is an organized detailed report for Ahmed Khaleel:
Examinee Data:
* Name: Ahmed Khaleel
* Date of Birth: 2/2/2016
* Gender: Male
* Nationality: Egyptian
* Language(s) spoken at home: Arabic, English
* Education level: High School Diploma
* Occupation: None (at the time of assessment)

Diagnostic Scale Results:
* ADHD Diagnostic Scale (ADHD-RS): 30/50 = 6.0 (mild symptoms of inattention and hyperactivity-impulsivity)
* Conners' Continuous Performance Task (CPT): 148/200 = 74% (average performance, with some omissions and errors)
* Wechsler Adult Intelligence Scale (WAIS): Verbal Comprehension Index (VCI): 95, Fluid Reasoning Index (FRI): 83, Processing Speed Index (PSI): 87 (average intelligence, with strengths in verbal comprehension and weaknesses in processing speed)
* Behavior Rating Inventory of Executive Function (BRIEF-A): Total score: 45 (moderate levels of executive function difficulties)

Executive Function Scale Results:
* Trail Making Tes

" Based on the provided context, here is an organized detailed report for Ahmed Khaleel:\nExaminee Data:\n* Name: Ahmed Khaleel\n* Date of Birth: 2/2/2016\n* Gender: Male\n* Nationality: Egyptian\n* Language(s) spoken at home: Arabic, English\n* Education level: High School Diploma\n* Occupation: None (at the time of assessment)\n\nDiagnostic Scale Results:\n* ADHD Diagnostic Scale (ADHD-RS): 30/50 = 6.0 (mild symptoms of inattention and hyperactivity-impulsivity)\n* Conners' Continuous Performance Task (CPT): 148/200 = 74% (average performance, with some omissions and errors)\n* Wechsler Adult Intelligence Scale (WAIS): Verbal Comprehension Index (VCI): 95, Fluid Reasoning Index (FRI): 83, Processing Speed Index (PSI): 87 (average intelligence, with strengths in verbal comprehension and weaknesses in processing speed)\n* Behavior Rating Inventory of Executive Function (BRIEF-A): Total score: 45 (moderate levels of executive function difficulties)\n\nExecutive Function Scale Results:\n

In [54]:
llm_chain.run(final_prompt)

Llama.generate: prefix-match hit


 Based on the provided context, here is a detailed report for the examinee "Ahmed Khaleel":
Examinee Data:
* Name: Ahmed Khaleel
* Date of Birth: 2/2/2016
* Gender: Male
* Ethnicity: Arabic
* Nationality: Egyptian
* Language(s) spoken at home: Arabic and English
* Religion: Islam
* Marital Status: Single
* Occupation: Student
* Education level: High School

Diagnostic Scale Results:

* Attention Deficit Hyperactivity Disorder (ADHD):
	+ Total Score: 72/82 = 87%
	+ Subscale scores:
		- Inattentive: 35/40 = 87.5%
		- Hyperactive Impulsive: 37/40 = 92.5%
		- Total Score: 72/82 = 87%
* Executive Function Scale:
	+ Total Score: 77/90 = 85.5%
	+ Subscale scores:
		- Shift: 34/40 = 85%
		- Emotional Control: 36/40 = 85%
		- Flexibility: 32/40 = 75%
		- Total Score: 77/90 = 85.5%

Learning Disability Assessment:
* Based on the results of the ADHD and Executive Function scales, it is unlikely that Ahmed has a significant learning disability. However, further assessment may be necessary to rule 

' Based on the provided context, here is a detailed report for the examinee "Ahmed Khaleel":\nExaminee Data:\n* Name: Ahmed Khaleel\n* Date of Birth: 2/2/2016\n* Gender: Male\n* Ethnicity: Arabic\n* Nationality: Egyptian\n* Language(s) spoken at home: Arabic and English\n* Religion: Islam\n* Marital Status: Single\n* Occupation: Student\n* Education level: High School\n\nDiagnostic Scale Results:\n\n* Attention Deficit Hyperactivity Disorder (ADHD):\n\t+ Total Score: 72/82 = 87%\n\t+ Subscale scores:\n\t\t- Inattentive: 35/40 = 87.5%\n\t\t- Hyperactive Impulsive: 37/40 = 92.5%\n\t\t- Total Score: 72/82 = 87%\n* Executive Function Scale:\n\t+ Total Score: 77/90 = 85.5%\n\t+ Subscale scores:\n\t\t- Shift: 34/40 = 85%\n\t\t- Emotional Control: 36/40 = 85%\n\t\t- Flexibility: 32/40 = 75%\n\t\t- Total Score: 77/90 = 85.5%\n\nLearning Disability Assessment:\n* Based on the results of the ADHD and Executive Function scales, it is unlikely that Ahmed has a significant learning disability. Howe